In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
# Optionally, check if the key is loaded
if "GOOGLE_API_KEY" not in os.environ:
    raise ValueError("GOOGLE_API_KEY not found in .env file.")

In [2]:
os.listdir("../../docs")

['requirements.md']

In [3]:
from crewai import LLM
# from crewai_tools import WebsiteSearchTool
# Read your API key from the environment variable
gemini_api_key = os.getenv("GOOGLE_API_KEY")

# Use Gemini 2.5 Pro Experimental model
gemini_llm = LLM(
    model='gemini/gemini-2.5-pro',
    api_key=gemini_api_key,
    temperature=0.0  # Lower temperature for more consistent results.
)



In [4]:
gemma_llm = LLM(
    model = "ollama/gemma3:270m",
    base_url="http://localhost:11434"
)

In [5]:
from crewai.tools import BaseTool

from crewai_tools import FileReadTool
read_requirements_tool = FileReadTool( "../../docs/requirements.md")  # File containing requirements

# Placeholder tool for fetching customer support data
class CustomerSupportDataTool(BaseTool):
    name: str = "Customer Support Data Fetcher"
    description: str = (
      "Fetches recent customer support interactions, tickets, and feedback. "
      "Returns a summary string.")

    def _run(self, argument: str) -> str:
        # In a real scenario, this would query a database or API.
        # For this example, return simulated data.
        print(f"--- Fetching data for query: {argument} ---")
        return (
            """Recent Support Data Summary:
- 50 tickets related to 'login issues'. High resolution time (avg 48h).
- 30 tickets about 'billing discrepancies'. Mostly resolved within 12h.
- 20 tickets on 'feature requests'. Often closed without resolution.
- Frequent feedback mentions 'confusing user interface' for password reset.
- High volume of calls related to 'account verification process'.
- Sentiment analysis shows growing frustration with 'login issues' resolution time.
- Support agent notes indicate difficulty reproducing 'login issues'."""
        )

support_data_tool = CustomerSupportDataTool()

/Users/yunweihu/miniconda/lib/python3.11/site-packages/pydantic/fields.py:1042: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warn(
/Users/yunweihu/miniconda/lib/python3.11/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)


Agents


In [6]:
from crewai import Agent

# Agent 1: function analyst
function_analyst = Agent(
    role='Functional Analyst',
    goal='Analyze Functional Requirements of the system of interest. Get ready for the next step of the process, which is to generate failuare modes for each function.',
    backstory=(
        """You are a Senior Systems Reliability Engineer with deep expertise in Failure Mode and Effects Analysis (FMEA) for {product}. Your primary role is to ensure that functional requirements are written with enough clarity, precision, and verifiability to serve as a solid foundation for identifying potential failure modes. You are an expert in applying the principles from the INCOSE Guide for Writing Requirements."""
    ),
    verbose=True,
    allow_delegation=False,  # This agent focuses on its specific task
    tools=[read_requirements_tool],  # Assign the data fetching tool
    llm=gemini_llm  # Use the configured Gemini LLM
)

failure_modes_analyst = Agent(
    role='Reliability and FMEA Analyst',
    goal='Generate a comprehensive and structured list of potential failure modes derived from a given set of technical functional requirements.',
    backstory=(
    "You are a Senior Reliability Engineer and a seasoned FMEA (Failure Mode and Effects Analysis) facilitator. "
    "With a deep background in {product} and systems engineering, you are an expert at dissecting functional requirements to anticipate every conceivable way a component or system could fail. Your work is meticulous, systematic, and forms the critical foundation for robust and safe engineering design. You live and breathe standards for risk analysis and reliability."
  ),
    verbose=True,
    allow_delegation=False,  # This agent focuses on its specific task
    tools=[],  # Assign the data fetching tool
    llm=gemini_llm  # Use the configured Gemini LLM
)

gemma_agent = Agent(
    role='document reviewer',
    goal='Review and analyze documents, fix grammatical errors, and ensure clarity and coherence in the text.',
    backstory=(
        "You are a highly skilled document reviewer with expertise in technical writing and editing. "
        "You task is only to review the documents generated by the other agents to ensure that they are free of grammatical errors. You will not generate any new content, but rather focus on improving the clarity and coherence of existing text. Your goal is to ensure that all documents are polished and professional, ready for final presentation or publication."
    ),
    llm = gemma_llm,
    )

review_agent = Agent(
    role='document reviewer',
    goal='Review and analyze documents, fix grammatical errors, and ensure clarity and coherence in the text.',
    backstory=(
        "You are a highly skilled document reviewer with expertise in technical writing and editing. "
        "You task is only to review the documents generated by the other agents to ensure that they are free of grammatical errors. You will not generate any new content, but rather focus on improving the clarity and coherence of existing text. Your goal is to ensure that all documents are polished and professional, ready for final presentation or publication."
    ),
    llm = gemini_llm,
    )

Tasks


In [ ]:
from crewai import Task

# Task 1: Analyze functional requirements
functional_analysis_task = Task(
    description=(
        """Fetch and analyze the functional requirements. Use the `read_requirements_tool`. Review each functional requirement individually and assess its readiness for an FMEA. For each requirement, perform the following analysis based on the core principles outlined in the INCOSE Guide for Writing Requirements:  
1. Assess Verifiability (C7): Is the requirement stated in a way that is measurable and testable? Can a clear pass/fail criterion be established? If not, explain what is missing.  
2. Assess Singularity/Atomicity (C5): Does the requirement address only one single function or characteristic? Identify any "and/or" conjunctions or phrases that combine multiple requirements into one statement. For the FMEA to be effective, we need to analyze the failure of one function at a time.  
3. Assess Unambiguity (C3): Is the language clear, precise, and free of vague terms? Identify any words like "approximately," "user-friendly," "robust," "flexible," or "without degradation" that could be interpreted in multiple ways. A failure mode must be derived from a precise function.  
4. Failure Mode Derivation: For each requirement, state whether a clear failure mode can be derived.  
- If YES: Provide an example of a primary failure mode (e.g., "Fails to maintain capacitance within tolerance," "Fails to withstand rated voltage").  
- If NO: Explain why a clear failure mode cannot be derived and suggest a specific improvement to the requirement's wording.  
5. Provide Recommendations: For any requirement that does not meet the criteria, provide a rewritten version that is atomic, verifiable, and unambiguous, making it FMEA-ready.  
6. Implement the recommendations: Update the list of functional requirements. Do not add or delete any requirements, just modify the existing ones to meet the criteria.
"""
    ),
    expected_output=(
        """Update the list of functional requirements. Do not add or delete any requireemnts. Output a markdown table with the following columns:
- Req. ID: The unique identifier for the functional requirement.
- Original Requirement: The original functional requirement text.
- Revised Requirement: The revised functional requirement text that meets the criteria.
- Verifiability: Assessment of whether the requirement is measurable and testable.
- Singularity/Atomicity: Assessment of whether the requirement is atomic and addresses a single function.
- Unambiguity: Assessment of whether the requirement is clear and free of vague terms.
- Failure Mode Derivation: Whether a clear failure mode can be derived from the requirement.
- Example Failure Mode: If applicable, provide an example of a primary failure mode derived from the requirement.
- Recommendations: Any suggestions for improving the requirement's wording to make it FMEA-ready."""
    ),
    agent=function_analyst  # Assign task to the data_analyst agent
)

failure_modes_generation = Task(
  description=(
    "You are conducting FMEA on {product}. "
    "Your task is to generate a comprehensive list of potential failure modes for each function listed in the requirements. "
    "A 'failure mode' is the specific manner in which a component can fail to meet its intended function. A failure mode describes what you observe when an item fails. It is the specific, observable manner of failure—the symptom. It is NOT a cause, or the undderlying reasons. A failure mode description should never include causal language like `due to,` `because of,` or `caused by.`\n\n"
    "**FRAMEWORK & GUIDE WORDS:**\n"
    "To ensure a thorough analysis, you MUST systematically apply the following generic failure mode categories as 'guide words' "
    "to each functional requirement. For each requirement, consider how it could fail in each of these ways:\n"
    "1.  **Loss of Function** (Total failure)\n"
    "2.  **Degraded Function** (Partial failure: Over Specification [Too much/high] or Under Specification [Too little/low])\n"
    "3.  **Intermittent Function**\n"
    "4.  **Unintended Function**\n"
    "5.  **Erroneous Timing** (Too late / Too early)\n"
    "6.  **Erroneous Duration** (Too long / Too short)\n"
    "7.  **Conditional Failure** (Fails only under specific, allowable operating conditions)\n\n"
    "** A failure mode Conditional Failure** (Fails only under specific, allowable operating conditions)\n\n"

    "**INSTRUCTIONS:**\n"
    "**Use the updated functional requirements provided in the context from the previous step**.  "
    "For each functional requirement, generate a list of specific, plausible failure modes. "
    "The failure modes you generate must be directly and technically related to the function described in the requirement. "
    "Be precise. For example, for FR-1, instead of just 'Degraded Function,' a specific failure mode would be "
    "'Capacitance is below the specified lower tolerance limit.' Apply all relevant guide words to each requirement."
  ),
  expected_output=(
    "A complete and detailed markdown table with four columns: 'Req. ID', 'Functional Requirement', 'Generic Failure Mode Category', and 'Specific Failure Mode(s)'. "
    "The table must be exhaustive, listing all plausible failure modes for EVERY provided requirement, categorized according to the given framework. "
    "The 'Specific Failure Mode(s)' column must contain clear, technical descriptions of the failure."
  ),
  agent=failure_modes_analyst,
  context=[functional_analysis_task],
  )

review_task = Task(
  description = " You are conducting a review to make sure the results generated are clear and do not have unneeded white space",
  expected_output=(
    "Updated markdown tables from previous tasks. The tables shall retain the original format, but all unnecessary white space are removed. "
  ),
  agent=review_agent,
  context=[failure_modes_generation],
)


Crew

In [8]:
from crewai import Crew, Process

# Define the crew with agents, tasks, and process
fmea_crew = Crew(
    agents=[function_analyst, failure_modes_analyst, review_agent],
    tasks=[functional_analysis_task, failure_modes_generation, review_task],
    process=Process.sequential,  # Tasks will run sequentially in the order defined
    verbose=True
)

Run

In [9]:
# Start the crew's work
print("--- Starting FMEA Advisor Crew ---")
# The 'inputs' dictionary provides initial context if needed by the first task.
# In this case, the tool simulates data fetching regardless of the input.
result =fmea_crew.kickoff(inputs={'product': 'Variable Frequency Drive (VFD)'})

print("--- Crew Execution Finished ---")
print("--- Final Report ---")


--- Starting FMEA Advisor Crew ---


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 24c7a068-5f7c-4f1e-ae24-82314024a03e                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Functional Analyst                                                                                      │
│                                                                                                                 │
│  Task: Fetch and analyze the functional requirements. Use the `read_requirements_tool`. Review each functional  │
│  requirement individually and assess its readiness for an FMEA. For each requirement, perform the following     │
│  analysis based on the core principles outlined in the INCOSE Guide for Writing Requirements:                   │
│  1. Assess Verifiability (C7): Is the requirement stated in a way that is measurable and testable? Can a clear  │
│  pass/fail criterion be established? If not, explain what is missing.                                           │
│  2. Assess Singularity/Atomicity (C5): Does the requirement address only one single function or                 │
│  characteristic? Identify any "and/or" conjunctions or phrases that combine multiple requirements into one      │
│  statement. For the FMEA to be effective, we need to analyze the failure of one function at a time.             │
│  3. Assess Unambiguity (C3): Is the language clear, precise, and free of vague terms? Identify any words like   │
│  "approximately," "user-friendly," "robust," "flexible," or "without degradation" that could be interpreted in  │
│  multiple ways. A failure mode must be derived from a precise function.                                         │
│  4. Failure Mode Derivation: For each requirement, state whether a clear failure mode can be derived.           │
│  - If YES: Provide an example of a primary failure mode (e.g., "Fails to maintain capacitance within            │
│  tolerance," "Fails to withstand rated voltage").                                                               │
│  - If NO: Explain why a clear failure mode cannot be derived and suggest a specific improvement to the          │
│  requirement's wording.                                                                                         │
│  5. Provide Recommendations: For any requirement that does not meet the criteria, provide a rewritten version   │
│  that is atomic, verifiable, and unambiguous, making it FMEA-ready.                                             │
│  6. Implement the recommendations: Update the list of functional requirements. Do not add or delete any         │
│  requirements, just modify the existing ones to meet the criteria.                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Functional Analyst                                                                                      │
│                                                                                                                 │
│  Thought: Thought: I need to start by fetching the functional requirements. I will use the `Read a file's       │
│  content` tool to read the `../../docs/requirements.md` file. This will give me the list of requirements to     │
│  analyze.                                                                                                       │
│                                                                                                                 │
│  Using Tool: Read a file's content                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: document reviewer                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ### Component FMEA: Potential Failure Modes for VFD Capacitor                                                  │
│                                                                                                                 │
│  The following table provides a comprehensive list of potential failure modes for the VFD capacitor, derived    │
│  systematically from the revised functional requirements. Each requirement has been analyzed against the        │
│  generic failure mode categories to identify specific, observable failure modes.                                │
│                                                                                                                 │
│  |Req. ID|Functional Requirement|Generic Failure Mode Category|Specific Failure Mode(s)|                        │
│  |:---|:---|:---|:---|                                                                                          │
│  |**FR-1**|The capacitor shall provide a nominal capacitance of 100 µF with a tolerance of ±5% when measured    │
│  at 60 Hz and 35°C.|**Loss of Function**|• Capacitor exhibits an open circuit (zero capacitance).<br>•          │
│  Capacitor exhibits a short circuit (infinite capacitance/zero impedance).|                                     │
│  |||**Degraded Function**|• Capacitance is below the specified lower tolerance limit (less than 95 µF). [Under  │
│  Specification]<br>• Capacitance is above the specified upper tolerance limit (greater than 105 µF). [Over      │
│  Specification]|                                                                                                │
│  |||**Intermittent Function**|• Capacitance value intermittently falls outside the specified ±5% tolerance      │
│  range.<br>• Capacitor intermittently shorts or opens.|                                                         │
│  |||**Unintended Function**|• Capacitor exhibits excessive parasitic inductance at operating frequencies.<br>•  │
│  Capacitor acts primarily as a resistor due to extremely high ESR.|                                             │
│  |||**Conditional Failure**|• Capacitance is within tolerance at room temperature but shifts out of tolerance   │
│  at 35°C.|                                                                                                      │
│  |**FR-2**|The capacitor shall have a rated DC working voltage of 5 VDC or greater.|**Loss of Function**|•      │
│  Dielectric breakdown (short circuit) occurs at a DC voltage below 5 VDC.|                                      │
│  |||**Degraded Function**|• Dielectric strength is significantly reduced, leading to premature failure under    │
│  transient conditions below 5 VDC.|                                                                             │
│  |||**Intermittent Function**|• Intermittent internal arcing occurs at a DC voltage below 5 VDC.|               │
│  |||**Unintended Function**|• Capacitor becomes a high-resistance path (partial short) at a DC voltage below 5  │
│  VDC.|                                                                                                          │
│  |||**Conditional Failure**|• Withstands 5 VDC at 25°C but experiences dielectric breakdown at 5 VDC when at    │
│  the maximum operating temperature (60°C).|                                                                     │
│  |**FR-3**|The capacitor shall withstand a continuous r

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 3efdb87a-abba-4543-bbde-f47d92788e44                                                                     │
│  Agent: document reviewer                                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 24c7a068-5f7c-4f1e-ae24-82314024a03e                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ### Component FMEA: Potential Failure Modes for VFD Capacitor                                    │
│                                                                                                                 │
│  The following table provides a comprehensive list of potential failure modes for the VFD capacitor, derived    │
│  systematically from the revised functional requirements. Each requirement has been analyzed against the        │
│  generic failure mode categories to identify specific, observable failure modes.                                │
│                                                                                                                 │
│  |Req. ID|Functional Requirement|Generic Failure Mode Category|Specific Failure Mode(s)|                        │
│  |:---|:---|:---|:---|                                                                                          │
│  |**FR-1**|The capacitor shall provide a nominal capacitance of 100 µF with a tolerance of ±5% when measured    │
│  at 60 Hz and 35°C.|**Loss of Function**|• Capacitor exhibits an open circuit (zero capacitance).<br>•          │
│  Capacitor exhibits a short circuit (infinite capacitance/zero impedance).|                                     │
│  |||**Degraded Function**|• Capacitance is below the specified lower tolerance limit (less than 95 µF). [Under  │
│  Specification]<br>• Capacitance is above the specified upper tolerance limit (greater than 105 µF). [Over      │
│  Specification]|                                                                                                │
│  |||**Intermittent Function**|• Capacitance value intermittently falls outside the specified ±5% tolerance      │
│  range.<br>• Capacitor intermittently shorts or opens.|                                                         │
│  |||**Unintended Function**|• Capacitor exhibits excessive parasitic inductance at operating frequencies.<br>•  │
│  Capacitor acts primarily as a resistor due to extremely high ESR.|                                             │
│  |||**Conditional Failure**|• Capacitance is within tolerance at room temperature but shifts out of tolerance   │
│  at 35°C.|                                                                                                      │
│  |**FR-2**|The capacitor shall have a rated DC working voltage of 5 VDC or greater.|**Loss of Function**|•      │
│  Dielectric breakdown (short circuit) occurs at a DC voltage below 5 VDC.|                                      │
│  |||**Degraded Function**|• Dielectric strength is significantly reduced, leading to premature failure under    │
│  transient conditions below 5 VDC.|                                                                             │
│  |||**Intermittent Function**|• Intermittent internal arcing occurs at a DC voltage below 5 VDC.|               │
│  |||**Unintended Function**|• Capacitor becomes a high-resistance path (partial short) at a DC voltage below 5  │
│  VDC.|                                                                                                          │
│  |||**Conditional Failure**|• Withstands 5 VDC at 25°C but experiences dielectric breakdown at 5 VDC when at    │
│  the maximum operating temperature (60°C).|           

--- Crew Execution Finished ---
--- Final Report ---


In [15]:
from IPython.display import display, Markdown

# Extract the raw text from the result object
if hasattr(result, "raw"):
    display(Markdown(result.raw))
else:
    raise TypeError("Result object does not contain raw text for Markdown.")

### Component FMEA: Potential Failure Modes for VFD Capacitor

The following table provides a comprehensive list of potential failure modes for the VFD capacitor, derived systematically from the revised functional requirements. Each requirement has been analyzed against the generic failure mode categories to identify specific, observable failure modes.

|Req. ID|Functional Requirement|Generic Failure Mode Category|Specific Failure Mode(s)|
|:---|:---|:---|:---|
|**FR-1**|The capacitor shall provide a nominal capacitance of 100 µF with a tolerance of ±5% when measured at 60 Hz and 35°C.|**Loss of Function**|• Capacitor exhibits an open circuit (zero capacitance).<br>• Capacitor exhibits a short circuit (infinite capacitance/zero impedance).|
|||**Degraded Function**|• Capacitance is below the specified lower tolerance limit (less than 95 µF). [Under Specification]<br>• Capacitance is above the specified upper tolerance limit (greater than 105 µF). [Over Specification]|
|||**Intermittent Function**|• Capacitance value intermittently falls outside the specified ±5% tolerance range.<br>• Capacitor intermittently shorts or opens.|
|||**Unintended Function**|• Capacitor exhibits excessive parasitic inductance at operating frequencies.<br>• Capacitor acts primarily as a resistor due to extremely high ESR.|
|||**Conditional Failure**|• Capacitance is within tolerance at room temperature but shifts out of tolerance at 35°C.|
|**FR-2**|The capacitor shall have a rated DC working voltage of 5 VDC or greater.|**Loss of Function**|• Dielectric breakdown (short circuit) occurs at a DC voltage below 5 VDC.|
|||**Degraded Function**|• Dielectric strength is significantly reduced, leading to premature failure under transient conditions below 5 VDC.|
|||**Intermittent Function**|• Intermittent internal arcing occurs at a DC voltage below 5 VDC.|
|||**Unintended Function**|• Capacitor becomes a high-resistance path (partial short) at a DC voltage below 5 VDC.|
|||**Conditional Failure**|• Withstands 5 VDC at 25°C but experiences dielectric breakdown at 5 VDC when at the maximum operating temperature (60°C).|
|**FR-3**|The capacitor shall withstand a continuous root-mean-square (RMS) ripple current of 0.1 A at 60 Hz and a maximum ambient temperature of 40°C without failure.|**Loss of Function**|• Catastrophic failure (vents, shorts, or opens) occurs at a ripple current below 0.1 A RMS.|
|||**Degraded Function**|• Internal temperature exceeds the capacitor's maximum rating when subjected to a ripple current less than 0.1 A RMS.<br>• Physical deformation (swelling, bulging) occurs at a ripple current below 0.1 A RMS.|
|||**Intermittent Function**|• Capacitor intermittently overheats when subjected to the specified ripple current.|
|||**Unintended Function**|• Generates audible noise when subjected to the specified ripple current.|
|||**Conditional Failure**|• Withstands 0.1 A RMS ripple current at 35°C but fails (e.g., overheats) at 40°C.|
|**FR-4**|The capacitor shall have a maximum Equivalent Series Resistance (ESR) of 100 mΩ when measured at 60 Hz and 35°C.|**Loss of Function**|• ESR is effectively infinite (internal connection open).|
|||**Degraded Function**|• ESR exceeds the 100 mΩ maximum limit. [Over Specification]|
|||**Intermittent Function**|• ESR value intermittently spikes above the 100 mΩ limit.|
|||**Conditional Failure**|• ESR is within the 100 mΩ limit at 25°C but exceeds it when measured at 35°C.|
|**FR-5**|The capacitor shall maintain all specified electrical characteristics (as defined in FR-1, FR-3, FR-4, FR-6) across an ambient operating temperature range of 0°C to 60°C.|**Loss of Function**|• Capacitor fails catastrophically (shorts, opens, vents) when operated at a temperature within the 0°C to 60°C range.|
|||**Degraded Function**|• Capacitance (FR-1) falls outside the ±5% tolerance when operated at a temperature within the 0°C to 60°C range.<br>• ESR (FR-4) exceeds 100 mΩ when operated at a temperature within the 0°C to 60°C range.<br>• Leakage current (FR-6) exceeds 10 µA when operated at a temperature within the 0°C to 60°C range.<br>• Fails to withstand ripple current (FR-3) when operated at a temperature within the 0°C to 60°C range.|
|||**Intermittent Function**|• Electrical characteristics become unstable or intermittent when operating near the temperature extremes (0°C or 60°C).|
|||**Unintended Function**|• Physical case cracks, swells, or leaks electrolyte when operated within the 0°C to 60°C range.|
|**FR-6**|The capacitor shall have a maximum leakage current of 10 µA after being subjected to its rated DC working voltage for 1 minute at 35°C.|**Loss of Function**|• Leakage current is excessively high, effectively creating a short circuit.|
|||**Degraded Function**|• Leakage current exceeds the 10 µA maximum limit under the specified test conditions. [Over Specification]|
|||**Intermittent Function**|• Leakage current intermittently spikes above the 10 µA limit under steady-state conditions.|
|||**Erroneous Timing**|• Leakage current takes significantly longer than 1 minute to stabilize below the 10 µA limit.|
|||**Conditional Failure**|• Leakage current is below 10 µA at 25°C but exceeds 10 µA at 35°C (under rated voltage).|
|**FR-7**|The capacitor's physical dimensions shall not exceed a diameter of 10 mm and a height of 20 mm.|**Degraded Function**|• Diameter exceeds 10 mm. [Over Specification]<br>• Height exceeds 20 mm. [Over Specification]|
|||**Conditional Failure**|• Dimensions are within specification at 25°C but expand beyond the maximum limits at the maximum operating temperature (60°C).|
|**FR-8**|The capacitor shall have a lead spacing of 5 mm ±0.5 mm.|**Degraded Function**|• Lead spacing is less than 4.5 mm. [Under Specification]<br>• Lead spacing is greater than 5.5 mm. [Over Specification]|
|**FR-9**|The capacitor shall have a minimum operational lifespan of 1,000 hours when operating continuously at the maximum rated ambient temperature (60°C), the rated DC working voltage (5 VDC), and the maximum rated ripple current (0.1 A RMS).|**Loss of Function**|• Catastrophic failure (open, short, vent) occurs before 1,000 hours under the specified life test conditions.|
|||**Degraded Function**|• A specified electrical parameter (e.g., capacitance, ESR, leakage current) drifts out of its specified tolerance before 1,000 hours under the life test conditions.|
|||**Erroneous Duration**|• Fails to operate for the required duration of 1,000 hours.|
|**FR-10**|The capacitor shall be compliant with the latest versions of both RoHS and REACH regulations.|**Loss of Function**|• Fails to comply with RoHS regulations (contains prohibited substances above threshold).<br>• Fails to comply with REACH regulations (contains Substances of Very High Concern (SVHCs) that are not declared).|

In [16]:
print(result.raw)


### Component FMEA: Potential Failure Modes for VFD Capacitor

The following table provides a comprehensive list of potential failure modes for the VFD capacitor, derived systematically from the revised functional requirements. Each requirement has been analyzed against the generic failure mode categories to identify specific, observable failure modes.

|Req. ID|Functional Requirement|Generic Failure Mode Category|Specific Failure Mode(s)|
|:---|:---|:---|:---|
|**FR-1**|The capacitor shall provide a nominal capacitance of 100 µF with a tolerance of ±5% when measured at 60 Hz and 35°C.|**Loss of Function**|• Capacitor exhibits an open circuit (zero capacitance).<br>• Capacitor exhibits a short circuit (infinite capacitance/zero impedance).|
|||**Degraded Function**|• Capacitance is below the specified lower tolerance limit (less than 95 µF). [Under Specification]<br>• Capacitance is above the specified upper tolerance limit (greater than 105 µF). [Over Specification]|
|||**Intermittent